**`This code shows how to use txtai to fine-tune the extractive QA model: 'ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA'.`**

**This code is an adaption from: https://github.com/motazsaad/Quran-QA by Basem Ahmed, Motaz Saad, and Eshrag A. Refaee**

Dataset: CGSQuAD

Dataset format: XLSX


* Dataset file name: CGSQuAD.xlsx
* Necessary columns: context, question, answers, answer_start, pq_id


#Install dependencies

In [ ]:
!pip install accelerate
!pip install git+https://github.com/SubaieiFatemah/txtai
!pip install datasets pandas
!pip install farasapy
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.7 MB/s eta 0:00:00
  Cloning https://github.com/SubaieiFatemah/txtai to /tmp/pip-req-build-ar2c0e5f
  Running command git clone --filter=blob:none --quiet https://github.com/SubaieiFatemah/txtai /tmp/pip-req-build-ar2c0e5f
  Resolved https://github.com/SubaieiFatemah/txtai to commit b44a9bcf7a64a7dec6f67f7204630ef62321449e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 84.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
  Created wheel for txtai: filename=txtai-5.6.0-py3-none-any.whl size=171766 s

In [ ]:
#!rm -r /content/Quran-QA
!git clone https://github.com/SubaieiFatemah/Quran-QA

Cloning into 'Quran-QA'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 143 (delta 61), reused 66 (delta 15), pack-reused 0
Receiving objects: 100% (143/143), 1.14 MiB | 14.41 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
!pip install accelerate
!pip install git+https://github.com/SubaieiFatemah/txtai
!pip install datasets pandas
!pip install farasapy
!pip install transformers

#!rm -r /content/Quran-QA
!git clone https://github.com/SubaieiFatemah/Quran-QA

import pandas as pd
import json
data = pd.read_excel('/content/CGSQuAD.xlsx')

train_df = data.sample(frac=0.8, random_state=42)
val_df = data.drop(train_df.index)

#dataset must be converted from Excel to JSONL format
train_jsonl_data = []
for _, row in train_df.iterrows():
    pq_id = str(row['pq_id'])
    question = str(row['question'])
    context = str(row['context'])
    answers = str(row['answers'])
    answer_start = int(row['answer_start'])#get answer_start from the df

    json_obj = {
        'pq_id': pq_id,
        'passage': context,
        'question': question,
        'answers': [{'text': answers, 'start_char': answer_start}]#list of dict
    }

    train_jsonl_data.append(json.dumps(json_obj, ensure_ascii=False))

with open('data_to_train.jsonl', 'w', encoding='utf-8') as train_jsonl_file:
    train_jsonl_file.write('\n'.join(train_jsonl_data))

val_jsonl_data = []
for _, row in val_df.iterrows():
    pq_id = str(row['pq_id'])
    question = str(row['question'])
    context = str(row['context'])
    answers = str(row['answers'])
    answer_start = int(row['answer_start'])

    json_obj = {
        'pq_id': pq_id,
        'passage': context,
        'question': question,
        'answers': [{'text': answers, 'start_char': answer_start}]
    }

    val_jsonl_data.append(json.dumps(json_obj, ensure_ascii=False))

with open('data_to_dev.jsonl', 'w', encoding='utf-8') as val_jsonl_file:
    val_jsonl_file.write('\n'.join(val_jsonl_data))

import sys
#sys.path.append("/content/")# another method but the order of checking directories does not matter
sys.path.insert(1, '/content/Quran-QA/quranqa/code/')
import read_write_qrcd
import quranqa22_eval

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from txtai.pipeline import HFTrainer
import torch, string, re, os, json
import operator

from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer

import importlib
importlib.reload(read_write_qrcd)#needed if the script is modified inside the session
train_passage_question_objects = read_write_qrcd.read_JSONL_file('/content/data_to_train.jsonl')
dev_passage_question_objects = read_write_qrcd.read_JSONL_file('/content/data_to_dev.jsonl')

train_data = [dict({
    "pq_id": passage_question_object.pq_id,
    "question": quranqa22_eval.normalize_text(passage_question_object.question),
    "context": passage_question_object.passage,
    "answers": r.text,
    "answer_start": train_df.loc[train_df['pq_id'] == passage_question_object.pq_id, 'answer_start'].iloc[0]
}) for passage_question_object in train_passage_question_objects
    for r in passage_question_object.answers]

dev_data = [dict({
    "pq_id": passage_question_object.pq_id,
    "question": quranqa22_eval.normalize_text(passage_question_object.question),
    "context": passage_question_object.passage,
    "answer_start": val_df.loc[val_df['pq_id'] == passage_question_object.pq_id, 'answer_start'].iloc[0]
}) for passage_question_object in dev_passage_question_objects
    for r in passage_question_object.answers]

def tokenize_function(row,tokenizer):
    return tokenizer.encode_plus(row['question'], row['context'],
                                 return_tensors='pt', padding=True,
                                 truncation=True,max_length=512,
                                 add_special_tokens = True)

topk_n = 20
def get_prediction(row, model, tokenizer):
    inputs = tokenize_function(row,tokenizer)
    output = model(**inputs)

    #print(torch.topk(output.start_logits.flatten(), topk_n))
    answer_starts_val = torch.topk(output.start_logits.flatten(), topk_n).values
    answer_ends_val = torch.topk(output.end_logits.flatten(), topk_n).values
    answer_starts = torch.topk(output.start_logits.flatten(), topk_n).indices
    answer_ends = torch.topk(output.end_logits.flatten(), topk_n).indices

    answers = []
    rank = 1
    for answer_start,answer_end,answer_start_v,answer_end_v in zip(answer_starts,answer_ends,answer_starts_val,answer_ends_val):
        answer = {}
        ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end+1]))
        if len(ans.strip())>0:
            answer["answer"] = ans
            answer["rank"] = rank
            answer["score"] = answer_start_v.item()+answer_end_v.item()
            rank = rank +1
            answers.append(answer)
        if rank == 4:
            break
    return answers

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    print(tokenizer.decode(prediction[0]))
    print(tokenizer.decode(prediction))
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    exact_match = compute_exact_match(pred, labels)
    f1 = compute_f1(pred, labels)

    return { "f1": f1, "exact_match": exact_match}

def test_model(tokenizer,model,data):
    result = {}
    for row in data:
        pred = get_prediction(row, model, tokenizer)
        result[row["pq_id"]] = pred
    return result


def Fine_tune(model_name,traindata,devdata,num_train_epochs,batch_size):
    trainer = HFTrainer()
    return trainer(model_name, traindata,#devdata,
                   task="question-answering",
                   metric_for_best_model = 'f1',
                   #evaluation_strategy ='epoch',
                   #eval_steps = 500, # Evaluation and Save happens every 50 steps
                   #save_total_limit = 15,
                   save_strategy='no',
                   learning_rate=3e-5,
                   per_device_train_batch_size=batch_size,
                   #per_device_eval_batch_size=batch_size,
                   num_train_epochs=num_train_epochs,
                   weight_decay=0.01,
                   #load_best_model_at_end=False, # change this to True after hyperparameter tuning
                   )


import ast
os.makedirs('Model2', exist_ok=True)
num_train_epochs=11
batch_size=8

for num_train_epochs in range(0, 2):
  model_name = "ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"
  model, tokenizer = Fine_tune(model_name, train_data, dev_data, num_train_epochs, batch_size)
  model.save_pretrained('Model2')
  tokenizer.save_pretrained('Model2')


import sys
import warnings
tokenizer = AutoTokenizer.from_pretrained('/content/Model2')
model = AutoModelForQuestionAnswering.from_pretrained('/content/Model2')
os.makedirs('preds', exist_ok=True)

result = test_model(tokenizer, model, dev_data)
open('/content/preds/zeyad_Run.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
metrics = !python /content/Quran-QA/quranqa/code/quranqa22_eval.py --run_file '/content/preds/zeyad_Run.json' --gold_answers_file '/content/data_to_dev.jsonl'
metrics = ast.literal_eval(metrics[-1])


#Read and convert CGSQuAD from XLSX into JSONL format

In [ ]:
import pandas as pd
import json
data = pd.read_excel('/content/CGSQuAD.xlsx')

train_df = data.sample(frac=0.8, random_state=42)
val_df = data.drop(train_df.index)

#dataset must be converted from Excel to JSONL format
train_jsonl_data = []
for _, row in train_df.iterrows():
    pq_id = str(row['pq_id'])
    question = str(row['question'])
    context = str(row['context'])
    answers = str(row['answers'])
    answer_start = int(row['answer_start'])#get answer_start from the df

    json_obj = {
        'pq_id': pq_id,
        'passage': context,
        'question': question,
        'answers': [{'text': answers, 'start_char': answer_start}]#list of dict
    }

    train_jsonl_data.append(json.dumps(json_obj, ensure_ascii=False))

with open('data_to_train.jsonl', 'w', encoding='utf-8') as train_jsonl_file:
    train_jsonl_file.write('\n'.join(train_jsonl_data))

val_jsonl_data = []
for _, row in val_df.iterrows():
    pq_id = str(row['pq_id'])
    question = str(row['question'])
    context = str(row['context'])
    answers = str(row['answers'])
    answer_start = int(row['answer_start'])

    json_obj = {
        'pq_id': pq_id,
        'passage': context,
        'question': question,
        'answers': [{'text': answers, 'start_char': answer_start}]
    }

    val_jsonl_data.append(json.dumps(json_obj, ensure_ascii=False))

with open('data_to_dev.jsonl', 'w', encoding='utf-8') as val_jsonl_file:
    val_jsonl_file.write('\n'.join(val_jsonl_data))

In [ ]:
data['context'][1000]

'اسكان طلبة الدراسات العليا هو اسكان الطلبة المسجلين بدوام كامل ممن لا تقيم اسرهم في الكويت في المساكن الداخلية. اسكان طلبة الدراسات العليا دون مقابل او رسوم. شروط اسكان طلبة الدراسات العليا ان يكون الطالب مسجلا بدوام كامل ولا تقيم اسرته في الكويت. نعم يجوز اسكان طلبة الدراسات العليا ممن تقيم اسرهم داخل الكويت في ضوء دراسة حالتهم الاجتماعية. شروط حضور المهمات العلمية لطالب الدراسات العليا ان يكون مسجلا ومستمرا كطالب نظامي بدوام كامل وان لا يقل معدله المتوسط عن 3.5 نقاط وقد اجتاز 15 وحدة دراسية على الاقل وبتوصية من مشرفه. عدد المهمات العلمية التي يستطيع الطالب حضورها هي مهمة واحدة فقط خلال مدة دراسته. نفقات المهمة العلمية هي نفقات الاشتراك وتذاكر سفر على الدرجة السياحية وبدل سفر يومي قدره 30 دينار كويتي. تسري احكام اللائحة اعتبارا من الفصل الاول للعام الجامعي التالي لاعتمادها من مجلس الجامعة. قيمة اشراف ومناقشة المشرف على المشروع هي 100 دينار كويتي للاشراف و100 دينار كويتي للمناقشة. قيمة مناقشة رئيس اللجنة على المشروع هي 100 دينار كويتي. قيمة مناقشة المناقش على المشروع هي 100 دينار كويت

In [ ]:
val_df.head()

,Unnamed: 0,pq_id,question,context,answers,answer_start
1,1.0,56be85543aeaaa14008c9065,ما هي غاية كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,اتاحة فرص تعليم,159
8,8.0,56d43c5f2ccc5a1400d830a9,ما العلاقة بين كلية الدراسات العليا والكليات ا...,رسالة كلية الدراسات العليا هي العمل المخطط اله...,علاقة عضوية,570
13,13.0,56d43ce42ccc5a1400d830b4,من هم اعضاء مجلس كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,عميد الكلية والعمداء المساعدون ورؤساء اللجان و...,757
14,14.0,56d43ce42ccc5a1400d830b5,مما يتكون مجلس كلية الدراسات العليا؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,عميد الكلية والعمداء المساعدون ورؤساء اللجان و...,757
20,20.0,56bf6e823aeaaa14008c962a,مما تتشكل لجنة المجال؟,رسالة كلية الدراسات العليا هي العمل المخطط اله...,مدراء البرامج بالكلية المعنية بالإضافة الى عدد...,1068


In [ ]:
val_jsonl_data[0]

'{"pq_id": "56be85543aeaaa14008c9065", "passage": "رسالة كلية الدراسات العليا هي العمل المخطط الهادف الى المساهمة في تنمية إمكانات جامعة الكويت لتكون مؤسسة علمية تعليمية متميزة. أهداف كلية الدراسات العليا تشمل اتاحة فرص تعليم ومواكبة التطور العالمي وصنع المثقف العصري وتناول قضايا المجتمع الكويتي. مهام كلية الدراسات العليا تشمل الموافقة على برامج الدراسات العليا ووضع الأنظمة وتحديد أسس القبول وعمل التقويم الدوري وطرح برامج اختصاصية. كلية الدراسات العليا هي المؤسسة المسؤولة عن الدراسات العليا في جامعة الكويت تخطيطا وتنفيذا وتطويرا. العلاقة بين كلية الدراسات العليا والكليات الأخرى ومراكز البحث العلمي داخل الجامعة هي علاقة عضوية تخضع للتنسيق الأكاديمي. مجلس كلية الدراسات العليا هو الهيئة العليا لبحث شؤون الكلية ووضع سياساتها والمسؤول عن الاشراف على البرامج وتقويمها. يتكون مجلس كلية الدراسات العليا من عميد الكلية والعمداء المساعدون ورؤساء اللجان وأعضاء هيئة التدريس وأعضاء يسميهم مدير الجامعة. عميد كلية الدراسات العليا هو المسؤول أكاديميا واداريا عن تنفيذ ومتابعة خطط البرامج وله أن يساعده عم

In [ ]:
import json

with open('data_to_train.jsonl', 'r') as file:
    for line in file:
        obj = json.loads(line)
        print(obj.keys())
        break

dict_keys(['pq_id', 'passage', 'question', 'answers'])


In [ ]:
import sys
#sys.path.append("/content/")# another method but the order of checking directories does not matter
sys.path.insert(1, '/content/Quran-QA/quranqa/code/')
import read_write_qrcd
import quranqa22_eval

from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from txtai.pipeline import HFTrainer
import torch, string, re, os, json
import operator

from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer

#Apply dataset script on CGSQuAD

The script converts the dataset into PassageQuestion objects.

In [ ]:
import importlib
importlib.reload(read_write_qrcd)#needed if the script is modified inside the session
train_passage_question_objects = read_write_qrcd.read_JSONL_file('/content/data_to_train.jsonl')
dev_passage_question_objects = read_write_qrcd.read_JSONL_file('/content/data_to_dev.jsonl')

Loaded 1203 records from /content/data_to_train.jsonl
pq_id: 56d440df2ccc5a1400d830d5
pq_id: 56cbf12a6d243a140015ee07
pq_id: 56d4bf242ccc5a1400d831c0
pq_id: 56cf6f874df3c31400b0d796
pq_id: 56bfed855a85de14001c7864
pq_id: 56cbf7d16d243a140015ee24
pq_id: 56cf67c74df3c31400b0d72d
pq_id: 56bfb1fca10cfb1400551254
pq_id: 56d4e9d12ccc5a1400d8333e
pq_id: 56bec9f13aeaaa14008c9469
pq_id: 56cf54a2aab44d1400b89009
pq_id: 56bf91c6a10cfb140055117f
pq_id: 56cbeb396d243a140015ede9
pq_id: 56d323fb59d6e41400146295
pq_id: 56bfb676a10cfb1400551269
pq_id: 56cc306b6d243a140015eec6
pq_id: 56beb0683aeaaa14008c9214
pq_id: 56becb8d3aeaaa14008c9498
pq_id: 56d4c4532ccc5a1400d83205
pq_id: 56cbd8c66d243a140015ed86
pq_id: 56d4e17f2ccc5a1400d832e2
pq_id: 56bfeb09a10cfb1400551394
pq_id: 56d4f63e2ccc5a1400d8338e
pq_id: 56bf9f6aa10cfb14005511d2
pq_id: 56cf64e34df3c31400b0d6f9
pq_id: 56cea7efaab44d1400b888f5
pq_id: 56d1cd98e7d4791d009021e7
pq_id: 56beb0683aeaaa14008c9213
pq_id: 56cc306b6d243a140015eec5
pq_id: 56cc44b76d2


Extract some attributes from PassageQuestion and Answer objects then save them as keys in a dictionary that represents the train and dev data.







In [ ]:
train_data = [dict({
    "pq_id": passage_question_object.pq_id,
    "question": quranqa22_eval.normalize_text(passage_question_object.question),
    "context": passage_question_object.passage,
    "answers": r.text,
    "answer_start": train_df.loc[train_df['pq_id'] == passage_question_object.pq_id, 'answer_start'].iloc[0]
}) for passage_question_object in train_passage_question_objects
    for r in passage_question_object.answers]

dev_data = [dict({
    "pq_id": passage_question_object.pq_id,
    "question": quranqa22_eval.normalize_text(passage_question_object.question),
    "context": passage_question_object.passage,
    "answer_start": val_df.loc[val_df['pq_id'] == passage_question_object.pq_id, 'answer_start'].iloc[0]
}) for passage_question_object in dev_passage_question_objects
    for r in passage_question_object.answers]

In [ ]:
train_data[0]

{'pq_id': '56d440df2ccc5a1400d830d5',
 'question': 'كيف تتألف لجنة البرنامج',
 'context': 'رسالة كلية الدراسات العليا هي العمل المخطط الهادف الى المساهمة في تنمية إمكانات جامعة الكويت لتكون مؤسسة علمية تعليمية متميزة. أهداف كلية الدراسات العليا تشمل اتاحة فرص تعليم ومواكبة التطور العالمي وصنع المثقف العصري وتناول قضايا المجتمع الكويتي. مهام كلية الدراسات العليا تشمل الموافقة على برامج الدراسات العليا ووضع الأنظمة وتحديد أسس القبول وعمل التقويم الدوري وطرح برامج اختصاصية. كلية الدراسات العليا هي المؤسسة المسؤولة عن الدراسات العليا في جامعة الكويت تخطيطا وتنفيذا وتطويرا. العلاقة بين كلية الدراسات العليا والكليات الأخرى ومراكز البحث العلمي داخل الجامعة هي علاقة عضوية تخضع للتنسيق الأكاديمي. مجلس كلية الدراسات العليا هو الهيئة العليا لبحث شؤون الكلية ووضع سياساتها والمسؤول عن الاشراف على البرامج وتقويمها. يتكون مجلس كلية الدراسات العليا من عميد الكلية والعمداء المساعدون ورؤساء اللجان وأعضاء هيئة التدريس وأعضاء يسميهم مدير الجامعة. عميد كلية الدراسات العليا هو المسؤول أكاديميا واداريا عن تن

#Create tokenizer and predictor functions

In [ ]:
def tokenize_function(row,tokenizer):
    return tokenizer.encode_plus(row['question'], row['context'],
                                 return_tensors='pt', padding=True,
                                 truncation=True,max_length=512,
                                 add_special_tokens = True)

topk_n = 20
def get_prediction(row, model, tokenizer):
    inputs = tokenize_function(row,tokenizer)
    output = model(**inputs)

    #print(torch.topk(output.start_logits.flatten(), topk_n))
    answer_starts_val = torch.topk(output.start_logits.flatten(), topk_n).values
    answer_ends_val = torch.topk(output.end_logits.flatten(), topk_n).values
    answer_starts = torch.topk(output.start_logits.flatten(), topk_n).indices
    answer_ends = torch.topk(output.end_logits.flatten(), topk_n).indices

    answers = []
    rank = 1
    for answer_start,answer_end,answer_start_v,answer_end_v in zip(answer_starts,answer_ends,answer_starts_val,answer_ends_val):
        answer = {}
        ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end+1]))
        if len(ans.strip())>0:
            answer["answer"] = ans
            answer["rank"] = rank
            answer["score"] = answer_start_v.item()+answer_end_v.item()
            rank = rank +1
            answers.append(answer)
        if rank == 4:
            break
    return answers


#Create evaluation, testing and training functions

In [ ]:
def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    print(tokenizer.decode(prediction[0]))
    print(tokenizer.decode(prediction))
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    exact_match = compute_exact_match(pred, labels)
    f1 = compute_f1(pred, labels)

    return { "f1": f1, "exact_match": exact_match}

In [ ]:
def test_model(tokenizer,model,data):
    result = {}
    for row in data:
        pred = get_prediction(row, model, tokenizer)
        result[row["pq_id"]] = pred
    return result

In [ ]:
def Fine_tune(model_name,traindata,devdata,num_train_epochs,batch_size):
    trainer = HFTrainer()
    return trainer(model_name, traindata,#devdata,
                   task="question-answering",
                   metric_for_best_model = 'f1',
                   #evaluation_strategy ='epoch',
                   #eval_steps = 500, # Evaluation and Save happens every 50 steps
                   #save_total_limit = 15,
                   save_strategy='no',
                   learning_rate=3e-5,
                   per_device_train_batch_size=batch_size,
                   #per_device_eval_batch_size=batch_size,
                   num_train_epochs=num_train_epochs,
                   weight_decay=0.01,
                   #load_best_model_at_end=False, # change this to True after hyperparameter tuning
                   )

#Fine-tune the model on CGSQuAD

In [ ]:
import ast
os.makedirs('Model2', exist_ok=True)
num_train_epochs=11
batch_size=8

for num_train_epochs in range(0, 2):
  model_name = "ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA"
  model, tokenizer = Fine_tune(model_name, train_data, dev_data, num_train_epochs, batch_size)
  model.save_pretrained('Model2')
  tokenizer.save_pretrained('Model2')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.296700
1000,0.821700
1500,0.731400
2000,0.731200
2500,0.691500
3000,0.679500
3500,0.683400
4000,0.672300
4500,0.671200


('Model2/tokenizer_config.json',
 'Model2/special_tokens_map.json',
 'Model2/vocab.txt',
 'Model2/added_tokens.json',
 'Model2/tokenizer.json')

#Test and evaluate the model on CGSQuAD

In [ ]:
import sys
import warnings
tokenizer = AutoTokenizer.from_pretrained('/content/Model2')
model = AutoModelForQuestionAnswering.from_pretrained('/content/Model2')
os.makedirs('preds', exist_ok=True)

result = test_model(tokenizer, model, dev_data)
open('/content/preds/zeyad_Run.json' , "w", encoding="utf8").write(json.dumps(result,indent=4, ensure_ascii=False))
metrics = !python /content/Quran-QA/quranqa/code/quranqa22_eval.py --run_file '/content/preds/zeyad_Run.json' --gold_answers_file '/content/data_to_dev.jsonl'
metrics = ast.literal_eval(metrics[-1])

Partial Reciprocal Rank (pRR): a metric used for evaluating information retrieval systems, usually used in QA and ranking tasks.

In [ ]:
metrics

{'pRR': 0.38564122351132984,
 'exact_match': 0.33554817275747506,
 'f1': 0.3811990996294092}

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("AraELECTRA-CGSQuAD-QA-Model2")
tokenizer.push_to_hub("AraELECTRA-CGSQuAD-QA-Tokenizer2")

CommitInfo(commit_url='https://huggingface.co/FatemahAlsubaiei/AraELECTRA-CGSQuAD-QA-Tokenizer2/commit/3a7e62264214dac9c2076abdbd0bf2a87bd11f79', commit_message='Upload tokenizer', commit_description='', oid='3a7e62264214dac9c2076abdbd0bf2a87bd11f79', pr_url=None, pr_revision=None, pr_num=None)